<a href="https://colab.research.google.com/github/PedroMarim/toxic-comment-bert/blob/main/notebooks/bert_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U "transformers" "datasets"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 7.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.2
    Uninstalling transformers-4.57.2:
      Successfully uninstalled transformers-4.57.2
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [ ]:
import transformers
print(transformers.__version__)
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import TrainingArguments, Trainer

4.57.2


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
%cd /content
!git clone https://github.com/PedroMarim/toxic-comment-bert.git
%cd toxic-comment-bert

/content
Cloning into 'toxic-comment-bert'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 11 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 27.92 KiB | 9.30 MiB/s, done.
/content/toxic-comment-bert


In [ ]:
import os

os.makedirs("data/raw", exist_ok=True)

In [ ]:
!pip install transformers datasets scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

label_cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

df = pd.read_csv("data/raw/train.csv")

df["any_toxic"] = (df[label_cols].sum(axis=1) > 0).astype(int)

train_df, temp_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["any_toxic"]
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    random_state=42,
    stratify=temp_df["any_toxic"]
)

len(train_df), len(val_df), len(test_df)


(127656, 15957, 15958)

In [ ]:
from datasets import Dataset
import numpy as np

label_cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

train_ds = Dataset.from_pandas(train_df[["comment_text"] + label_cols])
val_ds   = Dataset.from_pandas(val_df[["comment_text"] + label_cols])
test_ds  = Dataset.from_pandas(test_df[["comment_text"] + label_cols])


In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
max_length = 128

def tokenize_and_encode(examples):
    enc = tokenizer(
        examples["comment_text"],
        padding="max_length",
        truncation=True,
        max_length=max_length
    )
    # stack labels into shape (batch_size, num_labels)
    labels = np.stack([examples[label] for label in label_cols], axis=1)
    enc["labels"] = labels.astype(np.float32)
    return enc

train_encoded = train_ds.map(tokenize_and_encode, batched=True)
val_encoded   = val_ds.map(tokenize_and_encode, batched=True)
test_encoded  = test_ds.map(tokenize_and_encode, batched=True)

# Make them return PyTorch tensors
train_encoded.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_encoded.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_encoded.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/127656 [00:00<?, ? examples/s]

Map:   0%|          | 0/15957 [00:00<?, ? examples/s]

Map:   0%|          | 0/15958 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import BertForSequenceClassification

num_labels = len(label_cols)

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=num_labels,
    problem_type="multi_label_classification"
)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import roc_auc_score, classification_report
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))  # sigmoid

    # AUC (macro)
    try:
        auc_macro = roc_auc_score(labels, probs, average="macro")
    except ValueError:
        auc_macro = 0.0

    # F1 at 0.5 threshold
    preds = (probs >= 0.5).astype(int)
    report = classification_report(
        labels,
        preds,
        target_names=label_cols,
        output_dict=True,
        zero_division=0
    )
    macro_f1 = report["macro avg"]["f1-score"]
    micro_f1 = report["micro avg"]["f1-score"]

    return {
        "auc_macro": auc_macro,
        "macro_f1": macro_f1,
        "micro_f1": micro_f1
    }


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./models/bert-toxic",
    eval_strategy="epoch",          # with 4.57.3 this should work
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="auc_macro",
    greater_is_better=True,
    logging_steps=100,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encoded,
    eval_dataset=val_encoded,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-3085602259.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss,Auc Macro,Macro F1,Micro F1
1,0.035100,0.039323,0.984867,0.620048,0.777324
2,0.031400,0.039079,0.986292,0.665340,0.794282


TrainOutput(global_step=15958, training_loss=0.0388348541069724, metrics={'train_runtime': 5845.5637, 'train_samples_per_second': 43.676, 'train_steps_per_second': 2.73, 'total_flos': 1.6794455582416896e+16, 'train_loss': 0.0388348541069724, 'epoch': 2.0})

In [ ]:
metrics = trainer.evaluate(val_encoded)
metrics


{'eval_loss': 0.039078935980796814,
 'eval_auc_macro': 0.9862918820638605,
 'eval_macro_f1': 0.6653397228453377,
 'eval_micro_f1': 0.794282346006484,
 'eval_runtime': 108.0175,
 'eval_samples_per_second': 147.726,
 'eval_steps_per_second': 4.62,
 'epoch': 2.0}

In [ ]:
predictions = trainer.predict(test_encoded)
logits = predictions.predictions
probs = 1 / (1 + np.exp(-logits))
test_labels = predictions.label_ids

test_auc_macro = roc_auc_score(test_labels, probs, average="macro")
test_preds = (probs >= 0.5).astype(int)

print("Test AUC macro:", test_auc_macro)
print(
    classification_report(
        test_labels, test_preds, target_names=label_cols, zero_division=0
    )
)


Test AUC macro: 0.9895927757196993
               precision    recall  f1-score   support

        toxic       0.82      0.84      0.83      1541
 severe_toxic       0.58      0.39      0.47       167
      obscene       0.82      0.85      0.83       839
       threat       0.79      0.53      0.63        49
       insult       0.78      0.82      0.80       804
identity_hate       0.59      0.53      0.56       118

    micro avg       0.80      0.80      0.80      3518
    macro avg       0.73      0.66      0.69      3518
 weighted avg       0.79      0.80      0.79      3518
  samples avg       0.07      0.08      0.07      3518

